In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import numpy as np

# 1. Charger le dataset
df = pd.read_csv('tictactoe_dataset.csv')

# 2. Fonction pour convertir le plateau en vecteur numérique
def board_to_vector(board_str):
    mapping = {'X': 1, 'O': -1, ' ': 0}
    return [mapping[char] for char in board_str]

# Appliquer la conversion à la colonne 'board_state'
X = np.array([board_to_vector(board) for board in df['board_state']])

# 3. Encoder la colonne 'best_move' (notre cible y)
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df['best_move'])

print("Exemple de données transformées :")
print(f"Plateau original : '{df['board_state'][0]}'")
print(f"Plateau vectorisé (X) : {X[0]}")
print(f"Coup original : '{df['best_move'][0]}'")
print(f"Coup encodé (y) : {y[0]}")

Exemple de données transformées :
Plateau original : '         '
Plateau vectorisé (X) : [0 0 0 0 0 0 0 0 0]
Coup original : '0,0'
Coup encodé (y) : 0


In [3]:
# Diviser les données : 80% pour l'entraînement, 20% pour le test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTaille de l'ensemble d'entraînement : {X_train.shape[0]} échantillons")
print(f"Taille de l'ensemble de test : {X_test.shape[0]} échantillons")


Taille de l'ensemble d'entraînement : 3616 échantillons
Taille de l'ensemble de test : 904 échantillons


In [5]:
from sklearn.neighbors import KNeighborsClassifier

# Créer le modèle KNN
# On choisit k=5 voisins
knn = KNeighborsClassifier(n_neighbors=5)

# Entraîner le modèle avec les données d'entraînement
knn.fit(X_train, y_train)

print("\nLe modèle KNN a été entraîné avec succès ! ✅")


Le modèle KNN a été entraîné avec succès ! ✅


In [7]:
from sklearn.metrics import accuracy_score, classification_report

# Faire des prédictions sur l'ensemble de test
y_pred = knn.predict(X_test)

# Calculer la précision (accuracy)
accuracy = accuracy_score(y_test, y_pred)
print(f"\nPrécision du modèle : {accuracy:.4f}")

# Afficher un rapport de classification détaillé
print("\nRapport de Classification :")
# On utilise `label_encoder.classes_` pour afficher les vrais noms des coups
print(classification_report(y_test, y_pred, target_names=label_encoder.classes_))


Précision du modèle : 0.6825

Rapport de Classification :
              precision    recall  f1-score   support

         0,0       0.68      0.89      0.77       216
         0,1       0.58      0.66      0.62       117
         0,2       0.70      0.72      0.71       135
         1,0       0.63      0.59      0.61        73
         1,1       0.74      0.70      0.72       143
         1,2       0.67      0.45      0.54        49
         2,0       0.74      0.53      0.62        73
         2,1       0.67      0.41      0.51        34
         2,2       0.77      0.52      0.62        64

    accuracy                           0.68       904
   macro avg       0.69      0.61      0.63       904
weighted avg       0.69      0.68      0.68       904



In [9]:
# Exemple : un nouveau plateau de jeu
# X . .
# . O .
# . . .
new_board_str = 'X O      '

# 1. On doit le transformer de la même manière que les données d'entraînement
new_board_vector = np.array([board_to_vector(new_board_str)])

# 2. Faire une prédiction
predicted_move_encoded = knn.predict(new_board_vector)

# 3. Décoder la prédiction pour la rendre lisible
predicted_move = label_encoder.inverse_transform(predicted_move_encoded)

print(f"\nPour le plateau '{new_board_str}', le meilleur coup prédit est : {predicted_move[0]}")


Pour le plateau 'X O      ', le meilleur coup prédit est : 0,1


In [11]:
import joblib

# Sauvegarder le modèle entraîné
joblib.dump(knn, 'knn_model.joblib')

# Sauvegarder l'encodeur
joblib.dump(label_encoder, 'label_encoder.joblib')

print("Modèle KNN et LabelEncoder sauvegardés avec succès. ✅")

Modèle KNN et LabelEncoder sauvegardés avec succès. ✅
